In [1]:
import pandas as pd
import re 
import tweepy 
import time
import datetime
from tweepy import OAuthHandler
from langdetect import detect
from textblob import TextBlob
from IPython.display import clear_output
from langdetect.lang_detect_exception import LangDetectException
from operator import attrgetter
import time
from IPython.display import clear_output
import pixiedust

Pixiedust database opened successfully


In [2]:
#%%pixie_debugger
  
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXX'
        consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXX'
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
   
    '''
    Twitter only allows 100 tweets to be retrieved at a time.  This funciton
    operates recursively to get the number of tweets requested in query_count.
    '''
    def __get_tweets_recursive(self, query, count = 10, collected_tweets=None, max_id=None, tweets_needed=0): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = []
        ids=[]
        fetched_tweets=[]
        
        # check count
        if count < 10:
            tweets_needed = count
            count = 10
        
        query_count = count
        
        # clear_output()
        print ('>> count: {}'.format(count))
        print ('>> tweets_needed: {}'.format(tweets_needed))
        print ('>> max_id: {}'.format(max_id))
        try: 
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(tweet_mode="extended", max_id=max_id, q = query, count=query_count) 
            
            # use tweets_needed to query large number of tweets
            # even if we just need a few
            if not tweets_needed:
                tweets_needed = query_count
            
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                
                # keep track of ids for new queary 
                ids.append(tweet.id)
                
                # do a better check if tweet is english (first filtered in query but only to a point)
                try:
                    english = detect(tweet.full_text) == 'en'
                except LangDetectException as le:
                    # if anything other than english, assume not english
                    print ('Language Exception: {}. Ignoring Tweet'.format(le))
                    english = False
   
                if not english:
                    continue
                # only get long tweets
                if len(tweet.full_text) < 100:
                    continue
                else: 
                    tweets.append(tweet)
                    if len(tweets) == tweets_needed:
                        print('DONE! Last {} tweets retrieved.'.format(len(tweets)))
                        break
        
            # if did not get the number of tweets requested
            if len(tweets) < tweets_needed:
                new_query_count = int(tweets_needed) - len(tweets)
                # use this to get new tweets  
                max_id = min(ids)-1
                
                # this preserves the tweets_needed count for the next call
                tweets_needed = new_query_count
                
                # this helps avoid several small queries
                # query for more tweets than needed if need less than 10 
                if new_query_count < 10:
                    new_query_count = 10
                 
                print ('<< count: {}'.format(count))
                print ('<< new_query_count: {}'.format(new_query_count))
                print ('<< tweets_needed: {}'.format(tweets_needed))
                print ('<< max_id: {}'.format(max_id))
                print ('<< len(tweets): {}'.format(len(tweets)))
                print ('===================')
                more_tweets = self.__get_tweets_recursive(count=new_query_count,query=query,max_id=max_id, tweets_needed=tweets_needed)   
                
                # extends the list rather than appending the 
                # returned list object to the end of the list
                tweets.extend(more_tweets)

            # return parsed tweets
            print ('Returning {} tweets'.format(len(tweets)))
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 
            
        
    def remove_duplicate_tweets(self, list_of_tweets):
        
        tweets_no_dups = []
        seen_tweets_text = []
        
        for tweet in list_of_tweets:
            if tweet.full_text not in seen_tweets_text:
                tweets_no_dups.append(tweet)
                seen_tweets_text.append(tweet.full_text)
        
        return tweets_no_dups
        
    
    def get_tweets(self, query, count = 10, collected_tweets=None, max_id=None, tweets_needed=0): 
        
        tweets_check_for_dups = self.__get_tweets_recursive(query=query, 
                                                count = count, 
                                                collected_tweets = collected_tweets, 
                                                max_id = max_id, 
                                                tweets_needed = tweets_needed)
          
        tweets_no_dups = self.remove_duplicate_tweets(tweets_check_for_dups) 
        
        number_of_dups = len(tweets_check_for_dups) - len(tweets_no_dups)
       
        while number_of_dups > 0:
            
            print ('{} Duplicates Found'.format(number_of_dups))
            
            # get lowest tweet id
            min_num = min(tweets_check_for_dups,key=attrgetter('id'))
            max_id = min_num.id - 1
            
            # get tweets to replace the duplicates
            replace_dup_tweets = self.__get_tweets_recursive(query, count = count, 
                                                collected_tweets = collected_tweets, 
                                                max_id = max_id, 
                                                tweets_needed = number_of_dups)
            
            # add the replacement tweets to the list of unique tweets
            # replacements may have dups, will need to re-check
            tweets_check_for_dups = tweets_no_dups
            tweets_check_for_dups.extend(replace_dup_tweets)
            
            # check this new list for any duplicates
            tweets_no_dups = self.remove_duplicate_tweets(tweets_check_for_dups)
                    
            # if there are any more duplicates, go through the loop again
            number_of_dups = len(tweets_check_for_dups) - len(tweets_no_dups)

        
        return tweets_check_for_dups
        
        # Check for duplicate tweets, these could be across
        # multiple batches; better to just check the entire set rather
        # as we recursively grab them
        
        
        # Get remaining tweets starting from the lowest tweet retrieved
     
            
# creating object of TwitterClient Class 
api = TwitterClient() 
# calling function to get tweets



In [3]:
sarcastic_tweets = api.get_tweets(count = 1000 , query = '#sarcasm -filter:links -filter:media filter:safe -filter:retweets lang:en') 

>> count: 1000
>> tweets_needed: 0
>> max_id: None
<< count: 1000
<< new_query_count: 930
<< tweets_needed: 930
<< max_id: 1054147574930694143
<< len(tweets): 70
>> count: 930
>> tweets_needed: 930
>> max_id: 1054147574930694143
<< count: 930
<< new_query_count: 867
<< tweets_needed: 867
<< max_id: 1054030640730066944
<< len(tweets): 63
>> count: 867
>> tweets_needed: 867
>> max_id: 1054030640730066944
<< count: 867
<< new_query_count: 806
<< tweets_needed: 806
<< max_id: 1053797323350786048
<< len(tweets): 61
>> count: 806
>> tweets_needed: 806
>> max_id: 1053797323350786048
<< count: 806
<< new_query_count: 751
<< tweets_needed: 751
<< max_id: 1053659807419457536
<< len(tweets): 55
>> count: 751
>> tweets_needed: 751
>> max_id: 1053659807419457536
<< count: 751
<< new_query_count: 688
<< tweets_needed: 688
<< max_id: 1053447310179528705
<< len(tweets): 63
>> count: 688
>> tweets_needed: 688
>> max_id: 1053447310179528705
<< count: 688
<< new_query_count: 625
<< tweets_needed: 625
<< 

In [4]:
len(sarcastic_tweets)

1000

In [5]:
s_tweet_texts = []
s_tweet_ids = []
for s_tweet in sarcastic_tweets:
    s_tweet_texts.append(s_tweet.full_text)
    s_tweet_ids.append(s_tweet.id)   

In [6]:
df_st = pd.DataFrame()
df_st['id'] = s_tweet_ids
df_st['tweets']=s_tweet_texts
df_st['label'] = 1
df_st.head()

,id,tweets,label
0,1054369341704228869,@jeffrichadiha Which one of the 5 touchdowns d...,1
1,1054363559709327360,@SimmsYanne Everyone loves the DMV! I go there...,1
2,1054362726586187776,This no #sleep thing is my favorite. Its not i...,1
3,1054362006013140992,"@JamesKosur And today, the body double. I'm su...",1
4,1054359096629940224,Some people write TV coz they can't write Teli...,1


In [7]:
nonsarcastic_tweets = api.get_tweets(count = 1000, query = '-#sarcasm -#sarcastic -filter:links -filter:media filter:safe -filter:retweets lang:en') 

>> count: 1000
>> tweets_needed: 0
>> max_id: None
<< count: 1000
<< new_query_count: 963
<< tweets_needed: 963
<< max_id: 1054370517409521664
<< len(tweets): 37
>> count: 963
>> tweets_needed: 963
>> max_id: 1054370517409521664
<< count: 963
<< new_query_count: 926
<< tweets_needed: 926
<< max_id: 1054370515966664704
<< len(tweets): 37
>> count: 926
>> tweets_needed: 926
>> max_id: 1054370515966664704
<< count: 926
<< new_query_count: 899
<< tweets_needed: 899
<< max_id: 1054370514452561919
<< len(tweets): 27
>> count: 899
>> tweets_needed: 899
>> max_id: 1054370514452561919
<< count: 899
<< new_query_count: 875
<< tweets_needed: 875
<< max_id: 1054370512799854591
<< len(tweets): 24
>> count: 875
>> tweets_needed: 875
>> max_id: 1054370512799854591
<< count: 875
<< new_query_count: 847
<< tweets_needed: 847
<< max_id: 1054370511365394431
<< len(tweets): 28
>> count: 847
>> tweets_needed: 847
>> max_id: 1054370511365394431
<< count: 847
<< new_query_count: 817
<< tweets_needed: 817
<< 

DONE! Last 2 tweets retrieved.
Returning 2 tweets


In [8]:
ns_tweet_texts = []
ns_tweet_ids = []
for ns_tweet in nonsarcastic_tweets:
    ns_tweet_texts.append(ns_tweet.full_text)
    ns_tweet_ids.append(ns_tweet.id) 

df_nst = pd.DataFrame()
df_nst['id'] = ns_tweet_ids
df_nst['tweets'] = ns_tweet_texts
df_nst['label'] = 0
df_nst.head()

,id,tweets,label
0,1054370518969679872,@heydimpIes exactly thats why im like literall...,0
1,1054370518927843328,@KirenRijiju @narendramodi @Ra_THORe @BJP4Indi...,0
2,1054370518898540545,Cancel culture has only successfully canceled ...,0
3,1054370518860812288,I suddenly felt bad for all the mean things i ...,0
4,1054370518814490624,@V_of_Europe Canada is following Sweden with ...,0


In [9]:
df_st_nst = pd.concat([df_st, df_nst], ignore_index=True)

In [10]:
date_stamp = datetime.datetime.utcnow().strftime("%Y%m%d_%H%M")

In [11]:
file_name = 'sarcastic_' + str(len(df_st_nst)) + '_' + date_stamp + '.csv'

In [12]:
print (file_name)

sarcastic_2000_20181022_1356.csv


In [13]:
df_st_nst.to_csv(file_name,index=False)

# Combine Files

In [16]:
# Combine sarcastic file contents
import glob
allFiles = glob.glob('sarcastic_2000*.csv')
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)

/Users/christophersmyth/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [18]:
len(frame)

24000

In [27]:
frame.head()

,id,label,tweets
0,0.0,1,@mitchellreports @ChuckGrassley @SenKamalaHarr...
1,0.0,1,@paintingcorner You mean average followed by h...
2,0.0,1,@elizabethforma Probably should start handing ...
3,0.0,1,@SenBlumenthal In Roman times a loss like this...
4,0.0,1,I love #sarcasm just short of passive aggressi...


In [55]:
def remove_duplicate_records(tweet_df):

    tweets_no_dups = pd.DataFrame()
    seen_tweets_text = []
    dup_count = 0

    for index,row in tweet_df.iterrows():
        
        if row['tweets'] not in seen_tweets_text:
            seen_tweets_text.append(row['tweets'])
            tweets_no_dups = tweets_no_dups.append(row)
        else:
            dup_count += 1
            clear_output()
            print ('Duplicates: {}'.format(dup_count))
            print ('{}'.format(row['tweets']))
        

    return tweets_no_dups

In [56]:
nodups = pd.DataFrame()
nodups = remove_duplicate_records(frame)

Duplicates: 10173
I cast my Heisman ballot for #EdOliver! Click the player you think deserves the Heisman House vote. (📍@NissanUSA)


In [49]:
nodups.head()

,id,label,tweets
0,0.0,1.0,@mitchellreports @ChuckGrassley @SenKamalaHarr...
1,0.0,1.0,@paintingcorner You mean average followed by h...
2,0.0,1.0,@elizabethforma Probably should start handing ...
3,0.0,1.0,@SenBlumenthal In Roman times a loss like this...
4,0.0,1.0,I love #sarcasm just short of passive aggressi...


# Create an even file

In [60]:
sarc_only = pd.DataFrame()
sarc_only = nodups[nodups['label']==1]

In [61]:
sarc_len = len(sarc_only)
print (sarc_len)

2855


In [62]:
len(nodups[nodups['label']==0])

10972

In [63]:
nonsarc_even_sample = nodups[nodups['label']==0].sample(n=sarc_len, random_state=32)

In [64]:
even_sarc_nonsarc = sarc_only.append(nonsarc_even_sample)

In [65]:
len(even_sarc_nonsarc)

5710

In [67]:
date_stamp = datetime.datetime.utcnow().strftime("%Y%m%d_%H%M")
file_name = 'even_sarc_' + str(len(even_sarc_nonsarc)) + '_' + date_stamp + '.csv'
print (file_name)

even_sarc_5710_20181022_1524.csv


In [68]:
even_sarc_nonsarc.to_csv(file_name,index=False)